In [21]:
import requests, json, os
from climatiq import climatiq
import pandas as pd

CLIMATIQ_KEY = "J00J1VB0J9M1H8Q3TRP7GTTHXPFG"

with open("region-mapping.json") as f:
    REGION_MAPPING = json.load(f)

with open("azure_regions.json") as f:
    AZURE_REGIONS = json.load(f)

In [8]:
CLIMATIQ_BASE_URL = "https://beta3.api.climatiq.io"

def get_climatiq_api_key():
    if "CLIMATIQ_KEY" in globals():
        return CLIMATIQ_KEY
    elif "CLIMATIQ_KEY" in os.environ():
        return os.environ.get("CLIMATIQ_KEY")


def calculate_compute_emission(provider, region, cpu_count, cpu_load, duration, duration_unit):
    co2_data = get_co2_data()
    climatiq_api_key = get_climatiq_api_key()

    # normalise region
    region_mapping = REGION_MAPPING
    if region in region_mapping[provider].keys():
        region = region_mapping[provider][region]
    
    # Fetch data for calculation if not exists
    if not provider in co2_data["cpu"].keys():
        co2_data["cpu"][provider] = {}
        co2_data["cpu"][provider][region] = retrieve_climatiq_data(climatiq_api_key, provider, region)["co2e"]
        set_co2_data(co2_data)

    if not region in co2_data["cpu"][provider].keys():
        co2_data["cpu"][provider][region] = retrieve_climatiq_data(climatiq_api_key, provider, region)["co2e"]
        set_co2_data(co2_data)

    # Normalise to hourly units
    if duration_unit == "minute":
        duration = duration / 60
    elif duration_unit == "day":
        duration = duration * 24

    # Normalise cpu load if used in percentage
    if cpu_load > 1:
        cpu_load = cpu_load / 100

    return CO2_DATA["cpu"][provider][region] * cpu_count * cpu_load * duration


def retrieve_climatiq_data(api_key, provider, region, type="cpu", duration=1, duration_unit="h"):
    url = f"https://beta3.api.climatiq.io/compute/{provider}/{type}"
    headers = {"Authorization" : f"Bearer {api_key}"}

    data = {
        "cpu_count": 1,
        "region": region,
        "cpu_load": 1,
        "duration": int(duration),
        "duration_unit": duration_unit
    }

    res = requests.post(url, headers=headers, json=data)

    if not str(res.status_code)[0] == "2":
        raise Exception(f"Error calling climatiq API: {res.text}")

    return res.json()

In [17]:

def get_metadata(provider=""):
    url = f"https://beta3.api.climatiq.io/compute"
    headers = {"Authorization" : f"Bearer {CLIMATIQ_KEY}"}

    return requests.get(url, headers=headers)
    


providers = get_metadata().json()["cloud_providers"]

with open("provider_metadata.json", "w") as f:
    f.write(json.dumps(providers, indent=4))
    

In [68]:
provider_name = "azure"
instances = [i for i in providers[provider_name]["virtual_machine_instances"] if "4_" in i]

instance_data = []

for inst in instances:
    for provider_zone, climatiq_zone in REGION_MAPPING[provider_name].items():

        try:
            zone_details = list(filter(lambda x: x["name"] == provider_zone, AZURE_REGIONS))[0]
            
            item = {
            "provider" : provider_name,
            "dc_id" : provider_zone,
            "dc_cq_id" : climatiq_zone,
            "data center name" : zone_details["displayName"],
            "geo" : zone_details["metadata"],
            "activity_type" : "cpu",
            "activity_sub_type" : inst,
            "activity_sub_type_properties" : {"num_cpus": 4},
            "duration_int" : 1,
            "duration_type" : "hour",
            "emission_details" : retrieve_climatiq_data(CLIMATIQ_KEY, provider_name, climatiq_zone)
            }

            print(item)
            instance_data.append(item)

        except Exception as e:
            print(f"Failed to add data for zone: {provider_name} — {provider_zone}. With error {e}")
        



{'provider': 'azure', 'dc_id': 'australiacentral', 'dc_cq_id': 'australia_central', 'data center name': 'Australia Central', 'geo': {'geographyGroup': 'Asia Pacific', 'latitude': '-35.3075', 'longitude': '149.1244', 'pairedRegion': [{'id': '/subscriptions/1e663d00-d459-4c1d-88ba-7877566362e6/locations/australiacentral', 'name': 'australiacentral', 'subscriptionId': None}], 'physicalLocation': 'Canberra', 'regionCategory': 'Other', 'regionType': 'Physical'}, 'activity_type': 'cpu', 'activity_sub_type': 'e4_v3', 'activity_sub_type_properties': {'num_cpus': 4}, 'duration_int': 1, 'duration_type': 'hour', 'emission_details': {'co2e': 0.0020175, 'co2e_unit': 'kg', 'co2e_calculation_method': 'ar4', 'co2e_calculation_origin': 'source', 'emission_factor': {'activity_id': 'cpu-provider_azure-region_australia_central', 'uuid': '1a5aec93-9ec5-48b7-858a-6197fe6aad9e', 'id': 'cpu-provider_azure-region_australia_central', 'access_type': 'public', 'source': 'CCF', 'year': '2022', 'region': 'AU-CBR', 

In [69]:
with open("instance_data.json", "w") as f:
    f.write(json.dumps(instance_data, indent=4))

In [73]:
df = pd.read_json("instance_data.json")
df["lat"] = df["geo"].apply(lambda x: x["latitude"])
df["lon"] = df["geo"].apply(lambda x: x["longitude"])
df['co2e'] = df["emission_details"].apply(lambda x: x["co2e"])

In [105]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scattergeo(
        lon = df['lon'],
        lat = df['lat'],
        text = df['dc_cq_id'] + " — " + df['co2e'].astype('str'),
        mode = 'markers',
        marker = dict(
                color = df['co2e'],
                colorscale = "RdYlGn_r",
                size = 10,
                colorbar = dict(
                titleside = "right",
                outlinecolor = "rgba(68, 68, 68, 0)",
                ticks = "outside",
                showticksuffix = "last",
                dtick = 0.1
                        )
        )
        
        ))

fig.update_layout(
        title = 'CO2e per hour',
        width=1000,
        height=500,
        geo = dict(
                scope = 'world',
                showland = True,
                landcolor = "rgb(212, 212, 212)",
                subunitcolor = "rgb(255, 255, 255)",
                countrycolor = "rgb(255, 255, 255)",
                showlakes = True,
                lakecolor = "rgb(255, 255, 255)",
                showsubunits = True,
                showcountries = True,
                resolution = 50,
                projection = dict(
                        type = 'mercator',
                        rotation_lon = -100
        )
    ),
    )
fig.show()

In [125]:
import plotly.express as px
fig = px.scatter_geo(df, 
                lat="lat",
                lon="lon",
                color="co2e",
                hover_name="dc_cq_id", 
                size=[0.8]*len(df),
                projection="natural earth",
                color_continuous_scale = "RdYlGn_r",
                )
fig.show()
